# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# Get the orders data into dataframe

#data = pd.read_csv('Orders.csv')
data = pd.read_csv('Orders.zip', compression='zip', header=0, sep=',', quotechar='"')
data

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397919,541904,581587,22613,2011,12,5,12,pack of 20 spaceboy napkins,12,2011-12-09 12:50:00,0.85,12680,France,10.20
397920,541905,581587,22899,2011,12,5,12,children's apron dolly girl,6,2011-12-09 12:50:00,2.10,12680,France,12.60
397921,541906,581587,23254,2011,12,5,12,childrens cutlery dolly girl,4,2011-12-09 12:50:00,4.15,12680,France,16.60
397922,541907,581587,23255,2011,12,5,12,childrens cutlery circus parade,4,2011-12-09 12:50:00,4.15,12680,France,16.60


#### Standardizing column names 

In [3]:
# How to do this without replace? 
data.rename(columns = {'InvoiceNo':'Invoice_No', 'StockCode':'Stock_Code', 'InvoiceDate':'Invoice_Date', 'UnitPrice':'Unit_Price', 'CustomerID':'Customer_ID'}, inplace = True) # for making readable

data.columns = data.columns.str.replace('0', '').str.strip().str.lower().str.replace(' ', '_').str.replace(':', '')
data = data.reset_index(drop=True)
data = data.drop(['unnamed'], axis=1) 
data

,invoice_no,stock_code,year,month,day,hour,description,quantity,invoice_date,unit_price,customer_id,country,amount_spent
0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...
397919,581587,22613,2011,12,5,12,pack of 20 spaceboy napkins,12,2011-12-09 12:50:00,0.85,12680,France,10.20
397920,581587,22899,2011,12,5,12,children's apron dolly girl,6,2011-12-09 12:50:00,2.10,12680,France,12.60
397921,581587,23254,2011,12,5,12,childrens cutlery dolly girl,4,2011-12-09 12:50:00,4.15,12680,France,16.60
397922,581587,23255,2011,12,5,12,childrens cutlery circus parade,4,2011-12-09 12:50:00,4.15,12680,France,16.60


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

#### Sub Problem 1: Aggregate the  `amount_spent` for unique customers and write in new df

In [4]:
data2 = data['amount_spent'].groupby(by= data['customer_id'], axis=0, dropna=True).sum().reset_index()
data2

,customer_id,amount_spent
0,12346,77183.60
1,12347,4310.00
2,12348,1797.24
3,12349,1757.55
4,12350,334.40
...,...,...
4334,18280,180.60
4335,18281,80.82
4336,18282,178.05
4337,18283,2094.88


#### Sub Problem 2: Select customers whose aggregated `amount_spent` is in a given quantile range, which is 

#### VIP >= 95 %
#### Preferred >= 75 - <95 %

In [5]:
#quantiles = data2.drop(['customer_id'],axis=1).describe(percentiles=[0.75, 0.95]).drop(['count', 'mean' , 'std', 'min', '50%', 'max'],axis=0)

quantiles = data2['amount_spent'].quantile([0.75, 0.95])
quantiles

0.75    1661.640
0.95    5840.182
Name: amount_spent, dtype: float64

#### Sub Problem 3:  Label selected customers as "VIP" or "Preferred" and append to data frame

In [6]:
# Define a function to apply quantile range to each row
def label_customer(row):
    if row['amount_spent'] >= quantiles[0.95]:
        return 'VIP'
    elif row['amount_spent'] >= quantiles[0.75]:
        return 'Preferred'
    else:
        return np.nan

# Apply the function to each row to create the 'customer_class' column, and append result per row
data2['customer_class'] = data2.apply(label_customer, axis=1)
data2 = data2.dropna()
data2

,customer_id,amount_spent,customer_class
0,12346,77183.60,VIP
1,12347,4310.00,Preferred
2,12348,1797.24,Preferred
3,12349,1757.55,Preferred
5,12352,2506.04,Preferred
...,...,...,...
4319,18259,2338.60,Preferred
4320,18260,2643.20,Preferred
4328,18272,3078.58,Preferred
4337,18283,2094.88,Preferred


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

#### Cretae new DF country to merge with data2

In [7]:
# Get dataset by country
country = pd.DataFrame(data['country'], data['customer_id'].drop_duplicates()).reset_index()
#country = country.
country

,customer_id,country
0,17850,United Kingdom
1,13047,United Kingdom
2,12583,United Kingdom
3,13748,United Kingdom
4,15100,United Kingdom
...,...,...
4334,13436,United Kingdom
4335,15520,United Kingdom
4336,13298,United Kingdom
4337,14569,United Kingdom


In [8]:
data3 = data2.merge(country, left_on = 'customer_id', right_on = 'customer_id')
data3

,customer_id,amount_spent,customer_class,country
0,12346,77183.60,VIP,United Kingdom
1,12347,4310.00,Preferred,United Kingdom
2,12348,1797.24,Preferred,United Kingdom
3,12349,1757.55,Preferred,United Kingdom
4,12352,2506.04,Preferred,United Kingdom
...,...,...,...,...
1080,18259,2338.60,Preferred,United Kingdom
1081,18260,2643.20,Preferred,United Kingdom
1082,18272,3078.58,Preferred,United Kingdom
1083,18283,2094.88,Preferred,United Kingdom


#### Count the values country and show most frequent for VIP

In [9]:
data3[data3['customer_class'] == 'VIP'].groupby('country')['customer_class'].count().idxmax()

'United Kingdom'

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [10]:
#### what does it mean?? so without condition wich country occurs most frequent?

In [11]:
data3[data3['customer_class'].isin(['VIP', 'Preferred'])].groupby(['country', 'customer_class'])['customer_class'].count().sum(level='country').idxmax()

/var/folders/f3/3cnlgj7535533kpgd3y75z240000gn/T/ipykernel_32895/2825329007.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  data3[data3['customer_class'].isin(['VIP', 'Preferred'])].groupby(['country', 'customer_class'])['customer_class'].count().sum(level='country').idxmax()


'United Kingdom'

In [14]:
data3[data3['customer_class'] == 'VIP'].groupby('country')['customer_class'].count().idxmax()

data3[data3['customer_class'] == 'Preferred'].groupby('country')['customer_class'].count().idxmax()


'United Kingdom'